In [1]:
import numpy as np
import tensorflow as tf
import time
import random

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class Episode_experience():
    def __init__(self):
        self.memory = []
        
    def add(self, state, action, reward, next_state, done):
        self.memory += [(state, action, reward, next_state, done)]
        
    def clear(self):
        self.memory = []

class DDPGAgent:
    def __init__(self, state_size, action_size, action_low=-1, action_high=1, gamma=0.98,
                 actor_learning_rate=0.01, critic_learning_rate=0.01, tau=1e-3):
        self.state_size = state_size
        self.action_size = action_size
        self.action_low = action_low
        self.action_high = action_high
        self.gamma = gamma   # discount rate
        self.memory = []
        self.buffer_size = int(5e4)
        self.actor_learning_rate = actor_learning_rate
        self.critic_learning_rate = critic_learning_rate # often larger than actor_learning_rate
        self.tau = tau # soft update
        self.batch_size = 32
        self._construct_nets()
        
    def _construct_nets(self):
        tf.reset_default_graph()
        self.sess = tf.Session()
        
        self.S = tf.placeholder(tf.float32, [None, self.state_size], 'state')
        self.S_ = tf.placeholder(tf.float32, [None, self.state_size], 'next_state')
        self.D = tf.placeholder(tf.float32, [None, 1], 'done')
        self.R = tf.placeholder(tf.float32, [None, 1], 'r')

        with tf.variable_scope('Actor'):
            self.a = self._build_a(self.S, scope='eval')
            self.a_ = self._build_a(self.S_, scope='target')
        with tf.variable_scope('Critic'):
            q = self._build_c(self.S, self.a, scope='eval')
            q_ = self._build_c(self.S_, self.a_, scope='target')
        
        self.ae_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Actor/eval')
        self.at_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Actor/target')
        self.ce_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Critic/eval')
        self.ct_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Critic/target')

        self.soft_update_op = [[tf.assign(ta, (1 - self.tau) * ta + self.tau * ea), tf.assign(tc, (1 - self.tau) * tc + self.tau * ec)]
                             for ta, ea, tc, ec in zip(self.at_params, self.ae_params, self.ct_params, self.ce_params)]

        q_target = self.R + self.gamma * (1-self.D) * q_
        td_error_squared = tf.losses.mean_squared_error(q_target, q)
        self.ctrain = tf.train.AdamOptimizer(self.critic_learning_rate).minimize(td_error_squared, var_list=self.ce_params)
           
        a_loss = - tf.reduce_mean(q)    # maximize the q
        self.atrain = tf.train.AdamOptimizer(self.actor_learning_rate).minimize(a_loss, var_list=self.ae_params)
        
        self.sess.run(tf.global_variables_initializer())
    
    def _build_a(self, s, scope): # policy
        with tf.variable_scope(scope):
            net = tf.layers.dense(s, 64, tf.nn.relu, name='l1')
            a = tf.layers.dense(net, self.action_size, tf.nn.tanh, name='a')
            return a * (self.action_high-self.action_low)/2 + (self.action_high+self.action_low)/2
    
    def _build_c(self, s, a, scope): # Q value
        with tf.variable_scope(scope):
            net = tf.layers.dense(tf.concat([s, a], axis=1), 64, tf.nn.relu)
            return tf.layers.dense(net, 1)  # Q(s,a)
    
    def choose_action(self, state, variance): # normal distribution
        action = self.sess.run(self.a, {self.S: state})[0]
        return np.clip(np.random.normal(action, variance), self.action_low, self.action_high)
    
    def remember(self, ep_experience):
        self.memory += ep_experience.memory
        if len(self.memory) > self.buffer_size:
            self.memory = self.memory[-self.buffer_size:] # empty the first memories
        
    def replay(self, optimization_steps=1):
        if len(self.memory) < self.batch_size: # if there's no enough transitions, do nothing
            return
        
        for _ in range(optimization_steps):
            minibatch = np.vstack(random.sample(self.memory, self.batch_size))
            ss = np.vstack(minibatch[:,0])
            acs = np.vstack(minibatch[:,1])
            rs = np.vstack(minibatch[:,2])
            nss = np.vstack(minibatch[:,3])
            ds = np.vstack(minibatch[:,4])
            self.sess.run(self.atrain, {self.S: ss})
            self.sess.run(self.ctrain, {self.S: ss, self.a: acs, self.R: rs,
                                        self.S_: nss, self.D: ds})
    
    def update_target_net(self):
        self.sess.run(self.soft_update_op)

In [3]:
import gym
env = gym.make('MountainCarContinuous-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [4]:
agent = DDPGAgent(2, 1, tau=0.9)

In [5]:
# no param noise
rs = [0.0]

ep_experience = Episode_experience()
state = env.reset()
variance = 3

start_time = time.time()
for t in range(int(1e5)):
    action = agent.choose_action([state], variance)
    next_state, reward, done, _ = env.step(action)
    ep_experience.add(state, action, reward, next_state, done)
    agent.remember(ep_experience)  
    ep_experience.clear()
    rs[-1]+=reward
    state = next_state
    if done:
        state = env.reset()
        rs += [0.0]
    if t%50==0:
        variance *= 0.9995
        agent.replay(20)
        agent.update_target_net()
    if done and len(rs)%10==0:
        print("steps", t, "mean r %.2f"%np.mean(rs[-101:-1]), "variance %.2f"%variance)
        
print("Training time : %.2f"%(time.time()-start_time), "s")

steps 8990 mean r -81.64 variance 2.74
steps 18755 mean r -74.52 variance 2.49
steps 25690 mean r -43.58 variance 2.32
steps 30633 mean r -16.68 variance 2.21
steps 33172 mean r 3.07 variance 2.15
steps 35326 mean r 16.66 variance 2.11
steps 37848 mean r 25.92 variance 2.05
steps 39642 mean r 33.54 variance 2.02
steps 41791 mean r 39.16 variance 1.97
steps 44175 mean r 43.44 variance 1.93
steps 46195 mean r 58.85 variance 1.89
steps 48182 mean r 74.15 variance 1.85
steps 50641 mean r 80.81 variance 1.81
steps 52441 mean r 83.34 variance 1.78
steps 54180 mean r 84.03 variance 1.74
steps 56156 mean r 84.22 variance 1.71
steps 57980 mean r 84.82 variance 1.68
steps 59543 mean r 85.08 variance 1.65
steps 61219 mean r 85.53 variance 1.63
steps 63015 mean r 86.07 variance 1.60
steps 65247 mean r 85.93 variance 1.56
steps 66931 mean r 86.22 variance 1.54
steps 68494 mean r 86.92 variance 1.51
steps 70143 mean r 87.07 variance 1.49
steps 72550 mean r 86.63 variance 1.45
steps 74101 mean r 86.9

In [6]:
actions = []
for _ in range(5):
    state = env.reset()
    r = 0
    while True:
#         env.render()
        action = agent.choose_action([state], 0)
        next_state, reward, done, _ = env.step(action)
        r += reward
        state = next_state
        if done:
            break
    print("reward :", r)

reward : 93.41152048430182
reward : 93.41018269084864
reward : 93.40853161914451
reward : 93.39988396559905
reward : 93.30464309507579
